In [210]:
from bs4 import BeautifulSoup
import requests
import time
from random import randint
from IPython.core.display import clear_output
from warnings import warn
import re
from itertools import repeat

In [137]:
import numpy as np
import pandas as pd

In [217]:
root_url = 'https://www.offiziellecharts.de/charts/single/for-date-'

pages = list(range(1546297200000, 1577746800001, 86400000*7))

#pages = list(range(1546297200000, 1546297200001+(86400000*14), 86400000*7))

In [219]:
len(pages)

53

In [220]:
artists = []
songs = []
labels = []
positions = []
start_dates = []
end_dates = []
sources = []

request = 0
start_time = time.time()

In [221]:
for page in pages:

    # Make a get request
    response = requests.get(root_url + str(page))
    sources.append(root_url + str(page))

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    request += 1
    elapsed_time = time.time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(request, request/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(request, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if request > 53:
        warn('Number of requests was greater than expected.')
        break

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Scrape start and end date of the week
    date = soup.find("span", class_='ch-header').text
    start_date = re.findall(r'(\d+\.\d+\.\d+)', date)[0]
    end_date = re.findall(r'(\d+\.\d+\.\d+)', date)[1]
    start_dates.extend(repeat(start_date, 100))
    end_dates.extend(repeat(end_date, 100))
    
    # Select all the 100 song containers from a single page
    containers = soup.find_all('tr', class_ = 'drill-down-link')
        
    # For every song of these 100 songs
    for container in containers:
         # Scrape the artist
        artist = container.find('span', class_ = 'info-artist').text
        artists.append(artist)

        # Scrape the song
        song = container.find('span', class_ = 'info-title').text
        songs.append(song)

        # Scrape the label
        label = container.find('span', class_ = 'info-label').text
        labels.append(label)

        # Scrape the chart position
        position = container.find('span', class_ = 'this-week').text
        positions.append(position)

Request:53; Frequency: 0.08336464621791605 requests/s


In [222]:
charts = pd.DataFrame({'artist': artists,
                       'song': songs,
                       'label': labels,
                       'position': positions,
                       'start_date': start_dates,
                       'end_date': end_dates
                      })

charts.head(10)

,artist,song,label,position,start_date,end_date
0,Capital Bra,Benzema,Universal Domestic Urban,1,28.12.2018,03.01.2019
1,Mariah Carey,All I Want For Christmas Is You,SME - Columbia,2,28.12.2018,03.01.2019
2,Wham!,Last Christmas,SME - Epic,3,28.12.2018,03.01.2019
3,Ava Max,Sweet But Psycho,Atlantic,4,28.12.2018,03.01.2019
4,Chris Rea,Driving Home For Christmas,Warner Music International,5,28.12.2018,03.01.2019
5,Azet & Zuna,Lelele,KMN Gang,6,28.12.2018,03.01.2019
6,Melanie Thornton,Wonderful Dream (Holidays Are Coming),X-Cell,7,28.12.2018,03.01.2019
7,Sido,Tausend Tattoos,Universal Domestic Urban,8,28.12.2018,03.01.2019
8,Shakin' Stevens,Merry Christmas Everyone,SME - Epic,9,28.12.2018,03.01.2019
9,John & Yoko / The Plastic Ono Band with The Ha...,Happy Xmas (War Is Over),EMI,10,28.12.2018,03.01.2019


In [225]:
charts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5300 entries, 0 to 5299
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   artist      5300 non-null   object
 1   song        5300 non-null   object
 2   label       5300 non-null   object
 3   position    5300 non-null   object
 4   start_date  5300 non-null   object
 5   end_date    5300 non-null   object
dtypes: object(6)
memory usage: 248.6+ KB
